In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part'):
    shutil.rmtree('./file/3.part')

os.makedirs('./file/3.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                title = html.xpath('//head/title/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                subtitle = html.xpath('//h2[@itemprop="brand"]/a/span[@itemprop="name"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                type_ = html.xpath('//dd[@data-product-parttype]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                application_summary = html.xpath('//meta[@name="description"]/@content')[0].strip().removeprefix(f'{subtitle.title()},{type_},') if html.xpath('//meta[@name="description"]/@content') else ''

                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_src = html.xpath('//li[contains(@class, "productView-thumbnail")]/a[@class="productView-thumbnail-link"]/@href')
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src.strip()
    
                # = = = = = = = = = = = = = = =

                dict_info = {}
                list_dict = json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0])
                for i, dict_ in zip(range(len(list_dict)), list_dict):
                    dict_info[str(i)] = {dict_['name'].strip(): dict_['value'].strip()}
    
                # = = = = = = = = = = = = = = =

                bundle = resp.text.split('window.__webpack_public_path__ = "')[1].split('";</script>')[0]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': type_,
                                         'Part Number': part_number,
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Application Summary': application_summary,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Info': json.dumps(dict_info),
                                         'Bundle': bundle}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(30):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part/{file}',
              f'''./file/3.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 99.09% - crawler_98 > 2.https://www.cardone.com/product/power-steering-assist-motor-1c-18010m
[剩余数量：0] - [当前时间：22:05:45]

输出ing...

总数量：4

[状态：ok，尝试次数：1] - 100.00% - crawler_99 > 1.https://www.cardone.com/product/power-steering-control-valve-28-6623
[剩余数量：0] - [当前时间：22:05:47]

[状态：ok，尝试次数：1] - 100.00% - crawler_99 > 2.https://www.cardone.com/product/power-steering-control-valve-28-6652
[剩余数量：0] - [当前时间：22:05:49]

[状态：ok，尝试次数：1] - 100.00% - crawler_99 > 3.https://www.cardone.com/product/power-steering-control-valve-28-6653
[剩余数量：0] - [当前时间：22:05:51]

[状态：ok，尝试次数：1] - 100.00% - crawler_99 > 4.https://www.cardone.com/product/power-steering-control-valve-28-6658
[剩余数量：0] - [当前时间：22:05:53]

输出ing...



Progress: 100%|█████████████████████████| 110/110 [00:00<00:00, 10177.43it/s]

Done ~
